In [1]:
import numpy as np
import scipy.special
import matplotlib.pyplot as plt

# Методы Монте-Карло

В лабораторной [про квадратурные формулы](Integration.ipynb) мы пытались посчитать объем четырехмерного шара
методами трапеций и Симпсона. Абсолютная погрешность этих метод имеет асимптотику $h^d$ при шаге решетки $h\to0$, 
где $d$ - порядок метода.
Число точек, в которых должно быть вычислено подинтегральное выражение, изменяется как $h^{-D}$, где $D=4$ - размерность пространства. 
Для больших размерностей $D$ это делает использование квадратурных формул почти невозможным.
Существует однако возможность использовать небольшое число случайно выбранных точек для оценки интеграла, вмество построения мелкой решетки.
Этот подход базируется на связи интегралов и средних значений случайных величин:
$$
E_p[f]=\int_{\Omega} f(x)p(x)dx,
$$
где $f$ случайная величина, заданная на множестве $\Omega$ элементарных событий с плотностью распределения $p$,
$E_p[f]$ обозначает среднее значение $f$ на этом распределении.
Если мы имеем возможность генерировать выборку значений $x_k$ случайной величины $X$, отвечающей $p$, 
то среднее значение оценивается статистикой:
$$
E_p[f] \approx M_N = \frac{1}{N}\sum_{n=1}^N f(x_n),
$$
что дает нам связь между интегралом и суммой по случайно выбранным точкам.
Методы интегрирования на основе случайных выборок обычно называют методами Монте-Карло. 

Среднее арифметическое $M_N$ также является случайной величиной,
дисперсия которой оценивается как 
$$
\sigma(M_n)^2 = \frac{\sigma_p(f)^2}{N},
$$
где $\sigma_p(f)^2$ обозначает дисперсию случайной величины $f$.
Согласно этой формуле ошибка оценивания интеграла средним арфиметическим убывает как $N^{-\frac12}$,
что довольно медленно, однако эта скорость не зависит от размерности пространства, 
что делает метод Монте-Карло предпочтительным для задач большой размерности.

Для вычисления обьема четырехмерного шара $B=\{x\in\mathbb R^4\colon |x|^2\leq 1\},$
мы возьмем случайную величину $X$ равномерно распределенную на кубе $\Omega=[-1,1]^{\times 4}$.
Воспользуемся очевидными тождествами:
$$\frac{V(B)}{V(\Omega)}=P\{X\in B\}=E[1_B],$$
где $1_B$ - характеристическая функция множества $B$:
$$
1_B(x)=\begin{cases}
1,&x\in B,\\
0,&x\notin B.
\end{cases}
$$

In [2]:
# Для семплирования разных распределений удобно использовать пакет numpy.random.
# Создадим выборку из N=10000 равномерно распределенных на Omega значений.
N = 100000 # Размер выборки
D = 6 # Размерность пространства
x = np.random.rand(N, D)*2-1
# Найдем значения, попадающие в шар B
is_x_in_B = np.sum(x**2, axis=1)<1
# Посчитаем число попавших в B значений
x_in_B_count = np.sum(is_x_in_B)
# Оцениваем вероятность попасти в шар
p_x_in_B = x_in_B_count / N
# Обьем куба 
V_Omega = 2 ** D
# Наконец искомый интеграл равен
V_B_estimated = p_x_in_B * V_Omega
# Аналитическое значение интеграла
V_B_analytic = np.power(np.pi, D/2) / scipy.special.gamma(D/2+1)
print("Exact volume", V_B_analytic)
print("Estimated volume", V_B_estimated)
print("Relative error", (V_B_estimated-V_B_analytic)/V_B_analytic)


Exact volume 5.167712780049969
Estimated volume 5.19872
Relative error 0.006000182531377178


## Задания:

1. Вычислите методом Монте-Карло объемы $D$-мерных шаров радиуса $R=1$ для $D=1\ldots 15$ с выборками размера $N=1\ldots 10^6$.
Постройте график зависимости средней квадратической относительной ошибки объема от $N$ для разных $D$. Оцените асимптотику ошибки при $N\to\infty$. 

1. Теоретически обоснуйте наблюдаемую зависимость средней квадратической ошибки от размера выборки. 

1. Как можно оценить предельную ошибку вычисления интеграла методом Монте-Карло? 

1. Сравните ошибку метода Монте-Карло с результатами лабораторной [про квадратурные формулы](Integration.ipynb). Для каких размерностей $D$ метод Монте-Карло оказывается точнее, чем вычисление через произведение составных формул Симпсона по каждой из координат?

# Уменьшение ошибки метода Монте-Карло

Предположим мы хотим вычислить интеграл

$$I=\int_{\Omega} f(x)dx,$$

методом Монте-Карло, для этого мы представляем его в виде среднего математического величины $f(x)/p(x)$,
на плотности распределения $p$:

$$
I = E_p[f/p] = \int_{\Omega}\frac{f(x)}{p(x)}p(x)dx.
$$

Пусть $x_n$ выборка случайной величины $X$ с плотностью $p$, тогда 
$$
I\approx M_N=\frac{1}{N}\sum_{n=1}^N \frac{f(x_n)}{p(x_n)}.
$$
Дисперсия $M_N$ выражает меру ошибки:
$$
\sigma(M_N)^2 = \frac{1}{N}\sigma_p(f/p)^2=\frac{1}{N}\int_{\Omega} \left(\frac{f(x)}{p(x)}-I\right)^2 p(x)dx.
$$
Можно видеть, что хотя для всех $p$ среднее значение $M_N$ совпадает с искомым интегралом $I$,
величина ошибка существенно зависит от выбора $p$.
Идельным является выбор $p$ пропорциональным функии $f$, но генерация произвольного распределения само по себе является сложной задачей.
Поэтому на практике желательно выбирать известное распределение, имеющую плотность как можно более близко повторяющую $f$.
Данный метод является вариацией [выборки по значимости](http://ru.wikipedia.org/wiki/Выборка_по_значимости).

Попробуем улучшить точность вычислений объема шара, отталкиваясь от более естественного распределения, чем равномерное на кубе.
Наш предыдущий подход давал точный ответ для $D=1$. 
Теперь мы будем опираться на известное нам значение площади круга, и получим метод, дающий точный ответ для $D=2$.
Заметим, что если мы имеем пару случайных величин $\mu$, $\nu$, равномерно распределенных на интервале $[0,1]$,
то мы можем получить случайную точку $(x,y)$, равномерно распределенную на круге $x^2+y^2\leq 1$:
$$
\begin{cases}
x=r\cos\phi,\\
y=r\sin\phi,\\
\end{cases}
\begin{cases}
\phi=2\pi\mu,\\
r=\sqrt{\nu}.\\
\end{cases}
$$
Плотность распределения для $(x,y)$ равна 
$$
p(x,y)=\frac{1}{\pi}\theta(x^2+y^2),\quad
\theta(a)=\begin{cases}1,& a<1\\0,&\text{в противном случае}\end{cases}
$$
Идеальным распределением для вычисления обьема многомерного шара было бы равномерное распределение на этом шаре.
В качестве приближения этого распределения мы берем произведение равномерных распределений на кругах:
$$
p(x)=p(x_1,x_2)p(x_3,x_4)\cdots
$$

In [3]:
# Сгенерируем приближенное распределение
DHalf = int(D/2)
phi = 2*np.pi*np.random.rand(N, DHalf)
r = np.sqrt(np.random.rand(N, DHalf))
x = np.empty((N,D))
x[:,:2*DHalf:2]  = r*np.cos(phi)
x[:,1:2*DHalf:2] = r*np.sin(phi)
if D%2==1: x[:,-1] = np.random.rand(N)*2-1
# Вычисляем плотность распределения
p = np.power(np.pi, -DHalf)
if D%2==1: p /= 2
# Вычисляем величину f/p
f = np.array(np.sum(x**2, axis=1)<1, dtype=np.float)
f_over_p = f / p
# Вычисляем интеграл
V_B_second = np.mean(f_over_p)
# Вычисляем ошибку
V_B_analytic = np.power(np.pi, D/2) / scipy.special.gamma(D/2+1)
print("Exact volume", V_B_analytic)
print("Estimated volume", V_B_second)
print("Relative error", (V_B_second-V_B_analytic)/V_B_analytic)


Exact volume 5.167712780049969
Estimated volume 5.135259543791254
Relative error -0.006280000000000251


## Задания:

1. Постройте график средней ошибки от размера выборки. Сравните с ошибкой прошлой попытки.

1. Теоретически оцените величину уменьшения ошибки при переходе ко второму методу.

1. Предложите свой способ уменьшения вариации оценки обьема шара по методу Монте-Карло.

## Замечание:

В методы выше тяжело было найти плотность распределения $p$, сгенерировать выборку величины с данным распределением часто проще.
Например, равномерное распределение на шаре $B$ можно получить из выборки $X_k$ равномерно распределенной на кубе $\Omega$ случайной величины,
отбрасывая все точки $X_k$, не лежащие в $B$, что является частным случаем метода [алгоритма Метрополиса-Гастингса](http://ru.wikipedia.org/wiki/Алгоритм_Метрополиса_—_Гастингса). Оказывается в ряде случаев достаточно уметь строить выборку для решения содержательных задач, как показано в следующем разделе.

# Модель Изинга

[Модель Изинга](https://en.wikipedia.org/wiki/Ising_model) является одной из самых простых моделей ферромагнетиков в статистической физике.
Ферромагнетик в этой модели описывается системой случайных величин $\sigma_n$, принимающих значения $\pm 1$,
соответстующих двум противоположным направлениям магнитного момента отдельного атома $n$.
Обычно атомы считаются оргинзованными в некоторую кристаллическую решетку, 
в самом простом случае, который мы будем обсуждать ниже, атомы образуют цепочку, т.е. $n\in\mathbb Z$.
Энергия системы включает в себя вклады обмена и взаимодействия с внешним магнитным полем $h_j$:
$$H[\sigma] = -\sum_{<i,j>}J_{ij}\sigma_i\sigma_j-\mu\sum_i H_j\sigma_j,$$
где $J_{ij}$ суть константы обмена между парой атомов $\langle i,j\rangle$,
каждая пара атомов в сумме присутствует один раз,
$\mu$ - величина магнитного момента атома.
Вероятность реализации каждого состояниия подчинена распределению Больцмана:
$$
P_\beta(\sigma)=Z_\beta^{-1}e^{-\beta H[\sigma]},
$$
где $\beta=(k_BT)^{-1}$, и статистическая сумма определена следующей суммой по всем состояниям $\sigma$:
$$
Z_\beta = \sum_\sigma e^{-\beta H[\sigma]}.
$$
Вместо бесконечной цепочки, мы будем рассматривать цепочку из $L$ атомов, концы которой соединены между собой,
т.е. заданы периодические граничные условия.
Мы будем использовать самый простой вариант модели, считая (1) внешнее поле однородным  $h_j=h\forall j$,
(2) обмен ненулевым только для блищайших соседей и независящим от положения атомов на цепочке:
$J_{j,j+1}=J>0\forall j$.

При анализе модели наибольший интерес представляет свободная энергия Гельмгольца
$$F=-k_BT\ln Z.$$
В одномерном случае она может быть найдена явно.
Действительно, 
$$Z_\beta=\sum_{\sigma_1}\cdots\sum_{\sigma_L}\prod_{n=1}^{N-1}T_{\sigma_n,\sigma_{n+1}},$$
где 
$$T_{\sigma,\sigma'}=\exp\left(\beta J\sigma\sigma'+\frac{\mu h}{2}(\sigma+\sigma')\right).$$
Если считать $\sigma=-1,1$ индексами матрицы
$$T=\begin{pmatrix}T_{-1,-1} & T_{-1,1}\\ T_{1,-1} & T_{1,1}\end{pmatrix}
=\begin{pmatrix}e^{\beta J-\beta\mu h} & e^{-\beta J}\\ e^{-\beta J} & e^{\beta J+\beta\mu h}\end{pmatrix},$$
то стат. сумму можно записать в матричном виде:
$$Z_\beta=\mathrm{Tr}\,T^N=\lambda_+^N+\lambda_{-}^N,$$
где $\lambda_\pm$ суть собственные значения матрицы $T$.
Известно довольно мало других решений для модели Изинга, поэтому представляет интерес численный расчет свободной энергии.

Вычисление стат. суммы в данном случае заключается в суммировании $2^L$ слагаемых, что может быть сделано явно.
Однако с ростом числа спинов $L$ время работы растет экспоненциально, что делает прямое суммирование применимым только для малых систем.
Поэтому стат. суммы часто считают методами Монте-Карло.
Выше мы вычисляли интеграл с помощью Монте-Карло, однако метод с таким же успехом может использоваться и для сумм.
Так в простейшем варианте мы можем зафиксировать семейство случайных величин $s_n$, принимающих значения $-1$ и $1$ с одинаковой вероятностью.
Тогда стат. сумму можно выразить через среднее значение на этом распределении:
$$Z_\beta = E[e^{-\beta H[s]}] \approx \frac{2^L}{N}\sum_{k=1}^N \exp\big(-\beta H[s^{(k)}]\big),$$
где $s^{(k)}$ выборка состояний системы, равномерно распределеных на всем пространстве состояний.

In [4]:
from dataclasses import dataclass

# Объект типа dataclass: https://docs.python.org/3/library/dataclasses.html
# В нем мы будем хранить параметры системы.
@dataclass
class Ising:
    J: float # Константа обмена
    muh: float # Магнитная индукция 
    L: int # Число спинов
    def energy(self, state):
        """"Энергия состояния `state`"""
        return -model.J*np.sum(state*np.roll(state, 1, axis=-1), axis=-1)-model.muh*np.sum(state, axis=-1)
    def all_states(self):
        """Возвращает все состояния системы"""
        # Для хранения одного спина можно было ограничиться одним битом, но для простоты мы будем хранить число со знаком.        
        states = np.moveaxis( np.indices((2,)*model.L), 0, -1 ).reshape((2**model.L, model.L)).astype(np.int8)
        return 2*states-1
    def random_states_uni(self, n):
        """Возвращает `n` случайных значений."""
        return np.random.randint(0, 2, n*model.L).reshape((n,model.L))
        
def partition_function_expl(model: Ising, beta: float) -> float:
    """Вычисляет стат. сумму одномерной модели Изинга по явной формуле."""
    ebj = np.exp(beta*model.J)
    ebb = np.exp(beta*model.muh)
    T = np.array([[ebj/ebb, 1/ebj], [1/ebj, ebj*ebb]])
    lambdas = np.linalg.eigvalsh(T)
    return np.sum(np.power(lambdas, model.L))

def partition_function_direct(model: Ising, beta: float) -> float:
    """Вычисляет стат. сумму одномерной модели Изинга по определению."""    
    energy = model.energy(model.all_states())
    return np.sum(np.exp(-beta*energy))

def partition_function_mc(model: Ising, beta:float, nsamples:int) -> float:
    """Оценивает стат. сумму одномерной модели Изинга по Монте-Карло."""
    states = model.random_states_uni(nsamples)
#     states = model.all_states()
    energy = model.energy(states)
    p = np.exp(-beta*energy)
    return np.mean(p)*2**model.L


model = Ising(J=1.0, muh=0.1, L=1)
beta = 1.
print(f"Model {model} beta {beta}")
print(f"(ln Z)/L =")
print(f"  explicit   : {np.log(partition_function_expl(model, beta))/model.L}")
if model.L<=20:
    print(f"  direct     : {np.log(partition_function_expl(model, beta))/model.L}")
nsamples = 10000
print(f"  Monte-Carlo: {np.log(partition_function_mc(model, beta, nsamples))/model.L} #samples {nsamples}")    
    

Model Ising(J=1.0, muh=0.1, L=1) beta 1.0
(ln Z)/L =
  explicit   : 1.6981388693815918
  direct     : 1.6981388693815918
  Monte-Carlo: 1.382921018526089 #samples 10000


## Задания.

1. Оцените точность вычисления $Z_\beta$ через собственные значения матрицы $T$. Как ведет себя погрешность при $\beta\to 0$,
$\beta\to\infty$ и $L\to\infty$?

1. Оптимизируйте вычисления для прямого суммирования. Для какого размера системы вы можете воспользоваться этой формулой?

1. Вычислите стат. сумму методом Монте-Карло со равномерно и независимо распределенными случайными величинами $s$. Оцените скорость сходимости метода. Для относительной погрешности $10^{-5}$ оцените число спинов, начиная с которого метод Монте-Карло оказывается предпочтительнее прямого суммирования, если стоимость вычислений измерять в числе вызовов функции энергии $H$. 

1. Попробуйте повысить скорость сходимость метода Монте-Карло. Например, рассмотрите [алгоритм Ванга-Ландау](https://en.wikipedia.org/wiki/Wang_and_Landau_algorithm).

1. Постройте график зависимости средней энергии от константы связи $J$, используя найденную численно оценку для стат. суммы. Сравните с теорией.

1. Постройте тот же график без вычисления стат. суммы, опираясь только на метод [Метрополиса-Гастингса](http://ru.wikipedia.org/wiki/Алгоритм_Метрополиса_—_Гастингса) для генерирования выборки состояний $\sigma$, подчиненных распределению Больцмана.

1. Постройте график зависимости намагниченности от приложенного поля, используя связь стат. суммы и намагниченности:
$$M=\frac{\partial \ln Z}{\partial \mu h},$$
и сравните с расчетом намагниченности по алгоритму Метрополиса-Гастингса.

1. В модели с цепочкой не наблюдается фазового перехода, однако на двухмерной решетке есть фазовый переход по температуре.
Постройте график зависимости средней намагниченности от температуры для двухмерной решетки спинов.
Найдите критическое значение температуры. Опишите состояния при температуре ниже и выше критической.

## Задания:

1. Убедитесь, что ваш код хорошо структуирован, в частности: (a) одни и те же действия не выполняются в разных частях кода; (б) все вычисления разбиты на функции, каждая из которых имеет четкое и понятное предназначение и легко читается; (с) все параметры указаны явно, каждый параметр задается ровно в одном месте.